
# Testaus
- Molempien mallien syvällisempi analyysi ja käyttö 
- Kokeilu lähestymistapa ja tulosten kriittinen arviointi.

## 1. Puheentunnistus Whisper-mallilla

#### Testaus
transcribe_with_whisper() tallennettiin puheentunnistus.ipynb:stä tiedostoon saved_functions/transcribe_function.py, ja sen voi ajaa erikseen tarpeen mukaan.

In [2]:
import os
import glob
import librosa
from IPython.display import Audio, display
from saved_functions.transcribe_function import transcribe_with_whisper  

# Store results from Whisper
transcription_results = []
file_names = []

test_audiot = glob.glob("samples/test/*.wav")
model = "small"

for audio_path in test_audiot:
    y, sr = librosa.load(audio_path, sr=16000)
    print("Play the audio:")
    display(Audio(y, rate=sr))

    duration = librosa.get_duration(y=y, sr=sr)
    print(f"\nAudio Statistics:")
    print(f"Duration: {duration:.2f} seconds")
    print(f"Sample rate: {sr} Hz")

    file_name = os.path.basename(audio_path)
    file_names.append(file_name)
    
    print(f"\nTesting transcription for: {file_name}")
    result = transcribe_with_whisper(audio_path, model_size=model)
    print(f"\nTranscription ({file_name}):\n{result}")
    print("----------------------")
    transcription_results.append(result)

Play the audio:



Audio Statistics:
Duration: 5.64 seconds
Sample rate: 16000 Hz

Testing transcription for: normaali.wav
Using device: cpu
Loading Whisper small model...
Generating transcription...

Transcription (normaali.wav):
 Täällä on nyt lämmin ilmaa. Suomen kesä on kaunis.
----------------------
Play the audio:



Audio Statistics:
Duration: 4.05 seconds
Sample rate: 16000 Hz

Testing transcription for: puhekiltä.wav
Using device: cpu
Loading Whisper small model...
Generating transcription...

Transcription (puhekiltä.wav):
 Moi, masiointana kinkua.
----------------------
Play the audio:



Audio Statistics:
Duration: 22.44 seconds
Sample rate: 16000 Hz

Testing transcription for: vieraskieli.wav
Using device: cpu
Loading Whisper small model...
Generating transcription...

Transcription (vieraskieli.wav):
 Kevetä on Suomessa lyhyt. Huhtikuussa sataa joskus räntää ja on kylmä. Toukokuussa on jo aika lämmin. Puuut ovat vihreet ja puistoissa kukkivat tulppannit ja nassisit. Kese tulee.
----------------------
Play the audio:



Audio Statistics:
Duration: 8.21 seconds
Sample rate: 16000 Hz

Testing transcription for: sanaleikki.wav
Using device: cpu
Loading Whisper small model...
Generating transcription...

Transcription (sanaleikki.wav):
 66.66 juuralla kuinka monta kuusta kuusikkoissa kasvaa.
----------------------


In [3]:
from jiwer import wer

# Ground truths
ground_truth_list = [
    "Täällä on nyt lämmin ilmaa. Suomen kesä on kaunis.",
    "Moi, mä söi tänään kinkkua.",     
    "Kevät on Suomessa lyhyt. Huhtikuussa sataa joskus räntää ja on kylmä. Toukokuussa on jo aika lämmintä. Puut ovat vihreitä, ja puistoissa kukkivat tulppaanit ja narsissit. Kesä tulee.",
    "Kuusi kuusessa,kuusi kuusen juurella, kuinka monta kuusta kuusikkoissa kasvaa"
]

# Compute WER
print("\nWER Results:")
for i in range(len(ground_truth_list)):
    error_rate = wer(ground_truth_list[i], transcription_results[i])
    print(f"{file_names[i]} WER: {error_rate:.2f}")


WER Results:
normaali.wav WER: 0.00
puhekiltä.wav WER: 0.80
vieraskieli.wav WER: 0.26
sanaleikki.wav WER: 0.56


WER result = (virhesanat yhteensä) / (sanojen määrä ground truth -tekstissä)
1. normaali.wav: Täydellinen tulos – ei yhtään virhettä!
2. puhekieltä: On korkea virhemäärä – 80 % sanoista oli virheellisiä.
    - Mahdollisia syitä:
        - Epäselvä puhe(puhekieli)
        - Puhetyyli oli epämuodollinen (“mä söi” vs. “minä söin”)
3. vieraskieltä: Noin 26 % sanoista oli virheellisiä verrattuna oikeaan tekstiin.
    - Mahdollisia syitä:
        - Pitkä tai monimutkainen lause.
        - Epäselvää ääntämistä.
        - Pitkä tai monimutkainen lause.
4. sanaleikki.wav: 56% virheellistä.
    - mahdollista syitä:
        - Ei ymmärätänyt hyvin sanaleikistä.  